In [36]:
import os
import pandas as pd
from datetime import datetime

def starts_with_20(filename):
    """Check if the filename starts with '20'."""
    return filename.startswith("20")

def get_csv_files(folder_path):
    """Return a list of CSV files in the folder that start with '20'."""
    all_files = os.listdir(folder_path)
    csv_files = [file for file in all_files if file.endswith(".csv") and starts_with_20(file)]
    return csv_files

def read_and_combine_csv_files(folder_path, csv_files):
    """Read and concatenate the CSV files into a single DataFrame."""
    combined_dataframe = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in csv_files])
    return combined_dataframe

In [37]:
folder_path = "data/"
import numpy as np
import re

# Get a list of CSV files in the folder that start with "20"
csv_files = get_csv_files(folder_path)

# Read and concatenate the CSV files into a single DataFrame
combined_dataframe = read_and_combine_csv_files(folder_path, csv_files)

# Convert unix timestamp to datetime and consider the timezone, Brazil is UTC-3
combined_dataframe.dt = pd.to_datetime(combined_dataframe.dt, unit='s') - pd.Timedelta(days=1)
combined_dataframe.sunrise = pd.to_datetime(combined_dataframe.sunrise, unit='s') - pd.Timedelta(hours=3)
combined_dataframe.sunset = pd.to_datetime(combined_dataframe.sunset, unit='s') - pd.Timedelta(hours=3)

# Get only the time for sunrise and sunset
combined_dataframe.sunrise = combined_dataframe.sunrise.dt.time
combined_dataframe.sunset = combined_dataframe.sunset.dt.time


# Convert the weather column to dicts
combined_dataframe.weather = combined_dataframe.weather.apply(pd.Series)\
    .apply(lambda x: eval(x.to_list()[0])[0], axis=1)

# Convert the rain column to dicts


# Custom function to extract numbers
def extract_number(s):
    rain_mm_pattern = r'\d+\.\d+'
    if pd.isna(s):
        return np.nan
    else:
        matches = re.findall(rain_mm_pattern, s)
        return float(matches[0]) if matches else np.nan

# Apply the custom function to the DataFrame column
combined_dataframe.rain = combined_dataframe.rain.apply(extract_number)
combined_dataframe.rain.fillna(0.0, inplace=True)

# Drop duplicates
combined_dataframe.drop_duplicates(subset=['dt'], keep='first', inplace=True)

# Print the combined DataFrame
combined_dataframe

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,weather,rain,wind_gust,uvi
0,2011-12-31,05:22:24,18:56:12,20.15,20.60,1011,91,18.63,100,5000.0,3.60,110,"{'id': 701, 'main': 'Mist', 'description': 'mi...",0.66,NaN,NaN
1,2012-01-01,05:23:02,18:56:30,18.04,18.25,1010,90,16.38,100,8000.0,3.60,200,"{'id': 701, 'main': 'Mist', 'description': 'mi...",0.00,NaN,NaN
2,2012-01-02,05:23:40,18:56:47,18.39,18.30,1011,77,14.29,0,10000.0,2.10,160,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN
3,2012-01-03,05:24:19,18:57:02,19.91,19.94,1015,76,15.56,0,10000.0,3.10,180,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN
4,2012-01-04,05:24:58,18:57:17,20.22,20.26,1016,75,15.65,0,10000.0,4.10,120,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-03-26,06:12:19,18:12:07,23.27,23.92,1019,87,20.98,40,10000.0,8.49,182,"{'id': 501, 'main': 'Rain', 'description': 'mo...",3.65,10.28,0.0
86,2023-03-27,06:12:42,18:11:07,19.46,19.73,1021,87,17.24,75,10000.0,4.92,91,"{'id': 803, 'main': 'Clouds', 'description': '...",0.00,6.26,0.0
87,2023-03-28,06:13:04,18:10:08,20.75,21.20,1020,89,18.87,40,10000.0,4.12,110,"{'id': 802, 'main': 'Clouds', 'description': '...",0.00,NaN,0.0
88,2023-03-29,06:13:26,18:09:08,22.08,22.35,1020,77,17.86,0,10000.0,1.54,320,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,0.0


In [38]:
def find_complete_intervals(df, date_column):
    # Calculate the differences between consecutive dates
    df['Diff'] = df[date_column].diff().dt.days

    # Create a new column to group continuous intervals
    df['Group'] = (df['Diff'] > 1).cumsum()

    # Calculate start and end dates for each group
    intervals = df.groupby('Group')[date_column].agg(['min', 'max']).reset_index(drop=True)

    # Remove single date intervals
    intervals = intervals[intervals['max'] - intervals['min'] > pd.Timedelta(days=0)]

    # Remove auxiliar columns
    df.drop(['Diff', 'Group'], axis=1, inplace=True)

    return intervals


find_complete_intervals(combined_dataframe, 'dt')

,min,max
0,2011-12-31,2023-03-30


In [39]:
combined_dataframe["location"] = "Sao Paulo"
combined_dataframe

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,weather,rain,wind_gust,uvi,location
0,2011-12-31,05:22:24,18:56:12,20.15,20.60,1011,91,18.63,100,5000.0,3.60,110,"{'id': 701, 'main': 'Mist', 'description': 'mi...",0.66,NaN,NaN,Sao Paulo
1,2012-01-01,05:23:02,18:56:30,18.04,18.25,1010,90,16.38,100,8000.0,3.60,200,"{'id': 701, 'main': 'Mist', 'description': 'mi...",0.00,NaN,NaN,Sao Paulo
2,2012-01-02,05:23:40,18:56:47,18.39,18.30,1011,77,14.29,0,10000.0,2.10,160,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN,Sao Paulo
3,2012-01-03,05:24:19,18:57:02,19.91,19.94,1015,76,15.56,0,10000.0,3.10,180,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN,Sao Paulo
4,2012-01-04,05:24:58,18:57:17,20.22,20.26,1016,75,15.65,0,10000.0,4.10,120,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN,Sao Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-03-26,06:12:19,18:12:07,23.27,23.92,1019,87,20.98,40,10000.0,8.49,182,"{'id': 501, 'main': 'Rain', 'description': 'mo...",3.65,10.28,0.0,Sao Paulo
86,2023-03-27,06:12:42,18:11:07,19.46,19.73,1021,87,17.24,75,10000.0,4.92,91,"{'id': 803, 'main': 'Clouds', 'description': '...",0.00,6.26,0.0,Sao Paulo
87,2023-03-28,06:13:04,18:10:08,20.75,21.20,1020,89,18.87,40,10000.0,4.12,110,"{'id': 802, 'main': 'Clouds', 'description': '...",0.00,NaN,0.0,Sao Paulo
88,2023-03-29,06:13:26,18:09:08,22.08,22.35,1020,77,17.86,0,10000.0,1.54,320,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,0.0,Sao Paulo


In [40]:
# Extract all diferent weather conditions
wather_df = pd.DataFrame(combined_dataframe.weather.drop_duplicates().to_list())
wather_df

,id,main,description,icon
0,701,Mist,mist,50n
1,800,Clear,clear sky,01n
2,721,Haze,haze,50n
3,803,Clouds,broken clouds,04n
4,211,Thunderstorm,thunderstorm,11n
5,500,Rain,light rain,10n
6,501,Rain,moderate rain,10n
7,801,Clouds,few clouds,02n
8,802,Clouds,scattered clouds,03n
9,520,Rain,light intensity shower rain,09n


In [41]:
combined_dataframe["weather_id"] = combined_dataframe.weather.apply(lambda x: x['id'])
combined_dataframe

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,weather,rain,wind_gust,uvi,location,weather_id
0,2011-12-31,05:22:24,18:56:12,20.15,20.60,1011,91,18.63,100,5000.0,3.60,110,"{'id': 701, 'main': 'Mist', 'description': 'mi...",0.66,NaN,NaN,Sao Paulo,701
1,2012-01-01,05:23:02,18:56:30,18.04,18.25,1010,90,16.38,100,8000.0,3.60,200,"{'id': 701, 'main': 'Mist', 'description': 'mi...",0.00,NaN,NaN,Sao Paulo,701
2,2012-01-02,05:23:40,18:56:47,18.39,18.30,1011,77,14.29,0,10000.0,2.10,160,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN,Sao Paulo,800
3,2012-01-03,05:24:19,18:57:02,19.91,19.94,1015,76,15.56,0,10000.0,3.10,180,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN,Sao Paulo,800
4,2012-01-04,05:24:58,18:57:17,20.22,20.26,1016,75,15.65,0,10000.0,4.10,120,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,NaN,Sao Paulo,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-03-26,06:12:19,18:12:07,23.27,23.92,1019,87,20.98,40,10000.0,8.49,182,"{'id': 501, 'main': 'Rain', 'description': 'mo...",3.65,10.28,0.0,Sao Paulo,501
86,2023-03-27,06:12:42,18:11:07,19.46,19.73,1021,87,17.24,75,10000.0,4.92,91,"{'id': 803, 'main': 'Clouds', 'description': '...",0.00,6.26,0.0,Sao Paulo,803
87,2023-03-28,06:13:04,18:10:08,20.75,21.20,1020,89,18.87,40,10000.0,4.12,110,"{'id': 802, 'main': 'Clouds', 'description': '...",0.00,NaN,0.0,Sao Paulo,802
88,2023-03-29,06:13:26,18:09:08,22.08,22.35,1020,77,17.86,0,10000.0,1.54,320,"{'id': 800, 'main': 'Clear', 'description': 'c...",0.00,NaN,0.0,Sao Paulo,800


In [42]:
# Save data and weather conditions to CSV files

combined_dataframe.to_csv("data/combined_data.csv", index=False)
wather_df.to_csv("data/weather_conditions.csv", index=False)